In [4]:
pip install demoji

Note: you may need to restart the kernel to use updated packages.


In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import urllib.request
import os
import numpy as np
from emoji import core
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
from urllib.parse import quote_plus          
import time
from pathlib import Path
import os
from urllib.request import (urlopen, urlparse, urlunparse, urlretrieve)
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
import math
import re
from selenium.webdriver.chrome.service import Service
import os 
from webdriver_manager.chrome import ChromeDriverManager
from tqdm import tqdm
import pdb
import os
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import traceback         
from selenium.webdriver.common.proxy import Proxy, ProxyType
import csv
import requests
import json
import random
import psutil
from datetime import datetime
import pickle
import warnings
import demoji

In [2]:
def selenium_scroll_down(driver):
    SCROLL_PAUSE_SEC = 3
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_SEC)
        new_height = driver.execute_script("return document.body.scrollHeight")
  
        if new_height == last_height:
            return 1
        last_height = new_height


def get_driver(chrome_path, chrome_options, url):
    driver = None
    count = 0
    
    while (driver == None) and (count < 10):
            try:
                driver = webdriver.Chrome(chrome_path, options=chrome_options)
            except Exception:
                count = count + 1
                if driver: driver.quit()
                continue
    connect = False
    
    while connect == False: 
        try:
            driver.get(url)
            driver.implicitly_wait(10)
            connect = True
        except Exception:
            del driver
            driver = webdriver.Chrome(chrome_path, options=chrome_options)
            continue
    return driver

def reset_driver(driver, chrome_path, chrome_options, url, cookies):

    try :
        driver.quit()
        driver = get_driver(chrome_path, chrome_options, url, cookies)
    except Exception:
        driver = get_driver(chrome_path, chrome_options, url, cookies)
    return driver


def write_data(write_file, datas):

    for data in datas:
        write_file = write_file.append(data, ignore_index = True)
    
    return write_file

In [3]:
#카페 페이지 방문해서 카페 정보 가져오는 코드
def get_cafe_tag(driver):
    try:
        tags = WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.CLASS_NAME, 'txt_tag')))
        tags = tags.find_elements(By.CLASS_NAME, 'link_tag')
        return [tag.text for tag in tags]
    except Exception as e: 
        #print(e)
        return []

def get_cafe_menu(drvier):
    data = {}
    try:
        menu_list = WebDriverWait(driver, 2).until(EC.visibility_of_element_located((By.CLASS_NAME, 'list_menu')))
        menu_list = menu_list.find_elements(By.TAG_NAME, 'li')
        for menu in menu_list:
            menu = menu.find_element(By.CLASS_NAME, 'info_menu').text.split('\n')
            data[menu[0]] = menu[-1].replace(',','')
        return data
    except Exception as e: 
        #print('No menu info')
        return {}

In [4]:
#리뷰 영역 찾고 그 안에 정보 가져오는데 필요한 함수
def find_review_area(driver):
    try:
        reviews = WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.CLASS_NAME, 'evaluation_review')))
        return reviews
    except Exception as e: 
        #print(e)
        return None
def find_reviews(review_area):
    try: return review_area.find_elements(By.TAG_NAME, 'li')
    except Exception as e: 
        return None

#----------------------------------------------------------------------------------------------
def review_text(review, demoji):
    try: 
        text = review.find_element(By.CLASS_NAME, 'comment_info').text.replace('\n','. ')
       
        #emoji = demoji.findall(text)
        #if len(emoji) != 0: 
            #for em in emoji: text = text.replace(em,'')
        return text
    except Exception as e: 
        #print(e)
        return None
def star(review):
    try: 
        star = review.find_element(By.CLASS_NAME, 'ico_star.inner_star').get_attribute('style')
        return int(star.replace('width: ','').replace('%;',''))
    except: return None

#----main function---------------------------------------------------------------------------
def review_user_info(review, demoji):
    try: 
        user = review.find_element(By.CLASS_NAME, 'unit_info')
        user_info = user.text.split('\n')
        
        user_id = user_info[0]
        
        emoji = demoji.findall(user_id)
        if len(emoji) != 0: user_id = ''.join(emoji.values())
            
        review_count = int(user_info[2].replace(',',''))
        rating_avg = float(user_info[4])
        date = datetime.strptime(user_info[-1], '%Y.%m.%d.')
        return user_id, review_count, rating_avg, date
    except Exception as e: 
        print(e)
        return None, None, None, None
#------------sub function-------------------------------------------------------------------
def get_user_id(user):
    try: return user.find_element('link_user').text
    except: return None
def get_review_count(user):
    try: return int(user.find_element('txt_desc').text.replace(',',''))
    except: return None
def get_rating_avg(user):
    try: return float(user.find_element('txt_desc').text)
    except: return None
def get_date(user):
    try: return datetime.strptime(user.find_element('time_write').text, '%Y-%m-%d')
    except: return None
#----------------------------------------------------------------------------------------------

In [5]:
def concat_data(row, data):
    total = pd.merge(row, data, on = 'link', how = 'right')
    total['name'] = [row.name.values[0]] * len(total)
    total['category'] = [row.category.values[0]] * len(total)
    total['location'] = [row.location.values[0]] * len(total)
    total['rating'] = [row.rating.values[0]] * len(total)
    total['review_num'] = [row.review_num.values[0]] * len(total)
    total['rating_num'] = [row.rating_num.values[0]] * len(total)
    total['	open_hour'] = [row.open_hour.values[0]] * len(total)
    return total


In [6]:
def save(df,path, code, n):
    if n < len(code):
        unicode = code[n]
        try: 
            df.to_csv(path ,encoding = unicode, index=False)
            return True
        except Exception as e:
            print(e)
            return save(df, path, code, n+1)
    else: return False

In [7]:
def blog_title(driver):
    try: return driver.find_element(By.CLASS_NAME, 'pcol1').text
    except Exception as e:
        return None
def blog_date(driver):
    try: 
        date = driver.find_element(By.CLASS_NAME,'se_publishDate.pcol2').text
        return datetime.strptime(date, '%Y. %m. %d. %H:%M')
    except: return None
def blog_text(driver):
    try:  
        text = driver.find_element(By.CLASS_NAME,'se-main-container').text
        return text.replace('\n','.')
    except: return None
def blog_hashtag(driver):
    try: 
        hashtag = driver.find_element(By.CLASS_NAME,'wrap_tag').text.split('#')
        return str([tag.replace('\n','') for tag in hashtag])
    except: '[]'
        
def blog_writer(driver):
    try: return driver.find_element(By.CLASS_NAME,'writer').text
    except: return None
        
# 블로그 링크 접속 후 리뷰 크롤링
def blog_crawling(driver, link_list):

    blog_list = []
    title_list = []
    writer_list = []
    date_list = []
    content_list = []
    hashtag_list = []

    for link in link_list:
        data = {}
        driver.get(link)
        time.sleep(0.5)
        try:
            driver.switch_to.frame("mainFrame")
            blog_list.append(link)
            title_list.append(blog_title(driver))
            writer_list.append(blog_writer(driver))
            date_list.append(blog_date(driver))
            content_list.append(blog_text(driver))
            hashtag_list.append(blog_hashtag(driver))
        except Exception as e:
            print(e)
            
    blog_review = pd.DataFrame({'blog':blog_list,
                            'title':title_list,
                            'user_id':writer_list,
                            'date':date_list,
                            'contents':content_list,
                                'hashtag':hashtag_list})      
    return blog_review



    



# 블로그 리뷰 영역 추출
def get_blog_review_area(driver):
    try:
        return WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, 'cont_review')))
    except: None

# 블로그 리뷰 더보기 클릭
def blog_click_more(blog_area):
    try:
        button = WebDriverWait(blog_area, 5).until(EC.presence_of_element_located((By.CLASS_NAME, 'link_more')))
        if button.get_attribute("class") != 'link_more link_close':
            button.click()
            return True
        else: return False
    except Exception as e:
        return False

# 블로그 리뷰로 등록된 블로그 링크 추출
def get_blog_review_link(blog_area):
    link_list = []
    try:
        list_review = blog_area.find_element(By.CLASS_NAME, 'list_review').find_elements(By.TAG_NAME, 'li')
        for i in list_review:
            link = i.find_element(By.TAG_NAME, 'a').get_attribute('href')
            link_list.append(link)
        link_list = list(set(link_list))
        link_list = [i for i in link_list if i[:22]=='http://blog.naver.com/']
        link_num = round(len(link_list) * 0.2) + 1
        link_list = link_list[:link_num]
        return link_list         
    except:
        return None
    
    
# 최종 카카오맵 블로그 리뷰 크롤링 함수
def naver_blog_review(driver, link):
    
    blog_area = get_blog_review_area(driver)
    
    more = True
    while more == True:
        more = blog_click_more(blog_area)
        time.sleep(0.5)
    more = blog_click_more(blog_area)
    time.sleep(1)

    blog_area = get_blog_review_area(driver)
    link_list = get_blog_review_link(blog_area)
    if link_list: 
        blog_review = blog_crawling(driver, link_list)
        blog_review['link'] = [link] * len(blog_review)
        return blog_review, True
    else: return None, False
 

In [70]:
def click_more(review_area, actions):
    try:
        button = WebDriverWait(review_area, 5).until(EC.presence_of_element_located((By.CLASS_NAME, 'link_more')))
        if button.get_attribute("class") != 'link_more link_unfold':
            actions.move_to_element(button).perform()
            button.click()
            time.sleep(0.5)
            return True
        else: return False
    except Exception as e:
        return False
    


def crawl_data(driver, actions, demoji, row):
    try:
        url = row.link.values[0]
        driver.get(url)

        #hashtag = get_cafe_tag(driver)
        
    #카페 데이터(메뉴 정보 등) 얻기 ------------------------------------------------------------------
        menu = get_cafe_menu(driver)
        try:
            if 'Americano' in set(menu.keys()): row['Americano'] = int(menu['Americano'].replace(',',''))
            elif 'americano' in set(menu.keys()): row['Americano'] = int(menu['americano'].replace(',',''))
            elif '아메리카노' in set(menu.keys()): row['Americano'] = int(menu['아메리카노'].replace(',',''))
            elif '아이스커피' in set(menu.keys()): row['Americano'] = int(menu['아이스커피'].replace(',',''))
            elif '아이스 커피' in set(menu.keys()): row['Americano'] = int(menu['아이스 커피'].replace(',',''))
            elif '아메리카노(HOT)' in set(menu.keys()): row['Americano'] = int(menu['아메리카노(HOT)'].replace(',',''))
            elif '아메리카노(ICE)' in set(menu.keys()): row['Americano'] = int(menu['아메리카노(ICE)'].replace(',',''))

        except: row['Americano'] = None

        time.sleep(random.randrange(2))
    #카카오 리뷰 펼치기 ---------------------------------------------------------------------------
        review_area = find_review_area(driver)
        row['menu'] = str(menu)
        more = True
        while more == True:
            more = click_more(review_area, actions)
    #카카오 리뷰 긁기 ---------------------------------------------------------------------------
        kakao = False
        if row.rating_num.values[0] != None:
            if int(row.rating_num.values[0]) != 0:
                review_area = find_review_area(driver)
                if review_area != None:
                    review_area = review_area.find_element(By.CLASS_NAME, 'list_evaluation')
                    reviews = find_reviews(review_area)    
                    datas = []
                    #컬럼 이름 맞춰야함
                    cafe_data = pd.DataFrame(columns = ['link','user_id','user_review_count',
                                                        'user_rating_avg','star','date',
                                                       'title','contents','hashtag','blog'])
                    for review in reviews:
                        if review.get_attribute('data-id') != None:
                            data = {}
                            data['link'] = url
                            data['user_id'], data['user_review_count'], data['user_rating_avg'], data['date'] = review_user_info(review, demoji)
                            data['star'] = star(review)
                            data['contents'] = review_text(review, demoji)
                            data['title'] = None
                            data['hashtag'] = None
                            data['blog'] = None
                            datas.append(data)

                    cafe_data_kakao =  write_data(cafe_data, datas)
                    #카페 정보 + 리뷰 데이터 합니는 과정
                    cafe_data_kakao = concat_data(row, cafe_data_kakao)
                    kakao = True
                    print('kakao done')


    #네이버 리뷰 긁기 ---------------------------------------------------------------------------
        naver = False
        if row.review_num.values[0] != None:
            if (int(row.review_num.values[0]) != 0):
                cafe_data_naver, success = naver_blog_review(driver, url)
                if success:
                    cafe_data_naver = concat_data(row, cafe_data_naver)
                    naver = True
                    print('naver done')
        else: cafe_data_naver = None

    #전체 합치기 ---------------------------------------------------------------------------
        if naver and kakao:
            total = pd.concat([cafe_data_kakao, cafe_data_naver], axis = 0).reset_index()
            return total, True
        elif naver: return cafe_data_naver, True
        elif kakao: return cafe_data_kakao, True
        else: return 0, False
    except: return 0, False


In [71]:
def total_crawler(driver, cafe_list, visited):
    demoji.download_codes()
    actions = ActionChains(driver)
    code = ['utf-8-sig', 'utf-8','euc-kr','cp949','utf-16','utf-16-sig','utf-32','utf-32-sig']
    
    total = pd.read_csv('BDM/cafe/cafe_total1.csv')
    for i in tqdm(range(2286+140+6, len(cafe_list))):
        row = cafe_list.iloc[i:i+1,:]
        url = row.link.values[0]
        if (url not in visited):
            cafe_df, success = crawl_data(driver, actions, demoji, row)
            if success:
                if i == 2286+140+6: 
                    #total = cafe_df
                    total = pd.read_csv('BDM/cafe/cafe_total.csv')
                else:
                    total = pd.concat([total, cafe_df], axis = 0)
                    total.reset_index(drop=True, inplace=True)
                save(total,'BDM/cafe/cafe_total1.csv', code, 0)
        

In [10]:
#여기서 부터는 실행을 하기 위한 코드 순서대로 실행

In [18]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-setuid-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--disable-extensions")
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--allow-running-insecure-content')
chrome_options.add_argument("--single-process")
chrome_options.add_argument("disable-infobars")
#chrome_options.add_argument("--start-maximized")
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36'
chrome_options.add_argument(f'user-agent={user_agent}')
os.environ['WDM_LOG_LEVEL'] = '0'
os.environ['WDM_LOG'] = "false"
warnings.filterwarnings("ignore")

#본인 크롬 드라이버 위치한 파일 디렉토리 
chrome_path = 'C:\\Users\\dhkim\\IMG_Classifier\\chromedriver.exe'


In [69]:
driver =  get_driver(chrome_path, chrome_options, 'https://place.map.kakao.com/853028118')

In [10]:
#카페 리스트 파일 읽기(자기가 저장한 경로 알아서 입력)
cafe_list = pd.read_csv('BDM/cafe/cafe_info.csv')
cafe_list.rating_num = cafe_list.rating_num.fillna(0)
cafe_list.review_num = cafe_list.review_num.fillna(0)
cafe_list.drop_duplicates(inplace =True)
cafe_list.reset_index(inplace = True)

In [11]:
#괄호 안에 자기 번호 넣기
cafe_list = np.array_split(cafe_list, 5)[1]
#마지막으로 실행된 번호
cafe_list = cafe_list[1100:]
cafe_list = cafe_list.sample(frac = 1).reset_index(drop = True)

In [27]:
visited = pd.read_csv('BDM/cafe/cafe_total1.csv')['link']

In [28]:
visited = set(visited)

In [ ]:
 total_crawler(driver, cafe_list, visited)

  0%|          | 1/2074 [00:21<12:16:39, 21.32s/it]

kakao done


  0%|          | 2/2074 [01:10<21:51:47, 37.99s/it]

kakao done
naver done


  0%|          | 3/2074 [01:51<22:33:24, 39.21s/it]

...

kakao done
naver done


 13%|█▎        | 276/2074 [3:02:14<13:06:36, 26.25s/it]

kakao done


 13%|█▎        | 277/2074 [3:02:30<11:32:16, 23.11s/it]

kakao done


In [ ]:
1610